In [ ]:
import pandas as pd
import os
import numpy as np
import plotly
import plotly.express as px
import SimpleITK as sitk
import sys

sys.path.append("/mnt/famli_netapp_shared/C1_ML_Analysis/src/famli-ultra-sim/dl/")
from transforms.ultrasound_transforms import RealUSTrainTransforms

In [ ]:
df = pd.read_parquet("/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_c1_30082022.parquet")

In [ ]:
df.columns

In [ ]:
study_id_voluson = df.query('ManufacturerModelName == "Voluson S"')['study_id'].drop_duplicates()

In [ ]:
df_c = pd.read_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_Dataset_C_masked_resampled_256_spc075_wscores.parquet')

In [ ]:
study_id_butterfly = df_c[df_c['study_id'].isin(study_id_voluson)]['study_id'].drop_duplicates()

In [ ]:
study_id_voluson = study_id_voluson[study_id_voluson.isin(study_id_butterfly)].drop_duplicates()

In [ ]:
df_voluson = df.query('ManufacturerModelName == "Voluson S"')
df_voluson = df_voluson[df_voluson['study_id'].isin(study_id_butterfly)]

In [ ]:
df_butterfly = df_c[df_c['study_id'].isin(study_id_voluson)]

In [ ]:
study_id_voluson.sort_values().to_csv('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_study_id_voluson_butterfly.csv', index=False)

In [ ]:
df_voluson.to_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_voluson.parquet', index=False)

In [ ]:
df_butterfly.to_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_butterfly.parquet', index=False)

In [ ]:
df_t = pd.read_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_voluson_train_train.parquet')

In [ ]:
df_bt = pd.read_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_butterfly_train_test_50K.parquet')

In [ ]:
df_vt = pd.read_parquet('/mnt/raid/C1_ML_Analysis/CSV_files/extract_frames_blind_sweeps_voluson_train_test_50K.parquet')

In [ ]:
df_bt

In [ ]:
fn = os.path.join('/mnt/raid/C1_ML_Analysis', df_t['img_path'].loc[np.random.randint(len(df_t))])
img = sitk.ReadImage(fn)
img_np = sitk.GetArrayFromImage(img)
img_np = RealUSTrainTransforms()(img_np)
px.imshow(img_np[0].numpy())